In [45]:
# Library for accessing operation system functionality - environment variables here
import os
# Library used to load environment variables
from dotenv import load_dotenv
# OpenAI library
from openai import OpenAI
# Library to manage tokens
import tiktoken
# DateTime library
from datetime import datetime
# Library to work with JSON data
import json

# Load environment variables from .env file
load_dotenv()
# Access OpenAI's API Key stored in the .env file which is in the root folder
API_KEY = os.getenv('API_KEY')

# Set default values 
DEFAULT_MODEL = 'gpt-4o'
DEFAULT_TEMPERATURE = 0.2
DEFAULT_MAX_TOKENS = 512
DEFAULT_TOKEN_BUDGET = 4096

# Class to manage chatbot conversations 
# Includes session management, token management, persona setting, session management & persistence to files
class ChatManager():
    def __init__(self, api_key=None, model=None, temperature=None,max_tokens=None, token_budget=None,history_filepath=None):
        self.api_key = api_key if api_key else API_KEY
        self.model = model if model else DEFAULT_MODEL
        self.temperature = temperature if temperature else DEFAULT_TEMPERATURE
        self.max_tokens = max_tokens if max_tokens else DEFAULT_MAX_TOKENS
        self.token_budget = token_budget if token_budget else DEFAULT_TOKEN_BUDGET

        # Initiatialize list to track multi-turn conversations - prompts and responses
        self.messages = []
        # Method initializes a list that contains a predefined set of personas 
        self.set_system_messages()
        # Txt file to persist sessions at frequent checkpoints to ensure continuity
        self.history_filepath = history_filepath if history_filepath else self.get_history_filename()
        # Initialize OpenAI client
        self.client = OpenAI(api_key=self.api_key)


    def get_history_filepath(self):
        current_dt = datetime.now()
        formatted_dt = current_dt.strftime('%Y-%m-%d %H:%M:%S')
        filename = f"chat_history_{formatted_dt}"
        return filename

    def load_json_from_file(self):
      #  print(f'The history_filepath is {self.history_filepath}')
        try:
            with open(self.history_filepath, 'r') as file:
                data = json.load(file)
            return data 
        except FileNotFoundError:
            print(f"Error: The file '{self.history_filepath}' does not exist.")
        except json.JSONDecodeError:
            print(f"Error: The file '{self.history_filepath}' contains invalid JSON.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

    def load_conv_history(self):
        json_data = self.load_json_from_file()
        if json_data is not None:
           # print(json_data)
            self.messages = json_data
       
    def save_conv_history(self):
        try:
            with open(self.history_filepath,'w') as file:
                json.dump(self.messages, file, indent=4)
            print(f'Successfully saved data to {self.history_filepath}')
        except FileNotFoundError:
            print(f'Error: The file {self.history_filepath} doesnt exist')
        except IOError:
            print(f'Error: Could not read file {self.history_filepath}')
        except Exception as e:
            print(f'An unexpected error occurred: {e}')

    def set_system_messages(self):
        self.system_messages = {}
        self.system_messages['sassy_assistant'] = 'A sassy assistant who is fed up with answering questions'
        self.system_messages['angry_assistant'] = 'An angry assistant that likes yelling in all caps'
        self.system_messages['thoughtful_assistant'] = 'A thoughtful assistant, always ready to dig deeper. This assistant asks clarifying questions to ensure understanding and approaches problems with a step-by-step methodology.'
        self.system_messages['custom'] = ''

    def set_persona(self,persona):
        if persona in self.system_messages:
            print('yes persona is in the system messages')
            if len(self.messages) > 0:
                self.messages[0]['content'] = self.system_messages[persona]
            else:
                self.messages.append({'role':'system','content':self.system_messages[persona]})
         

    def set_custom_system_message(self, custom_persona_message):
        self.system_messages['custom'] = custom_persona_message
        print('yes I set the custom message')

    def count_tokens(self, text):
        #Initiatize the tokenizer
        tokenizer = tiktoken.encoding_for_model('gpt-4')
        #Tokenize the input text
        tokens = tokenizer.encode(text)
        #return the number of tokens
        return len(tokens)

    def total_tokens_used(self):
        total_tokens = 0
        for row in self.messages:
            total_tokens+=self.count_tokens(row['content'])
        return total_tokens

    def enforce_token_budget(self):
        try:
            while self.total_tokens_used() >= self.token_budget:
                if len(self.messages) > 1:
                    self.messages.pop(1)
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

    def chat_completion(self,prompt,persona):
        self.load_conv_history()
        self.set_persona(persona=persona)
        print(f"Total length of messages in conv history is {len(self.messages)}")
        self.messages.append({'role':'user','content':prompt})
        self.enforce_token_budget()
        print(f"Total tokens used so far is {self.total_tokens_used()}")
        response = self.client.chat.completions.create(seed=122, messages=self.messages,model=self.model, temperature=self.temperature, max_tokens = self.max_tokens)
        self.messages.append({'role':'assistant','content':response.choices[0].message.content})
        self.save_conv_history()
        return response.choices[0].message.content

In [47]:
conv_manager = ChatManager(temperature = 1.0,history_filepath='conv_history.txt')

In [49]:
response = conv_manager.chat_completion(prompt='Give me a status update on the outreach activity?', persona='thoughtful_assistant')

yes persona is in the system messages
Successfully saved data to conv_history.txt


In [31]:
print(response)

Certainly! Here's a general template for an outreach activity status update. You can customize it with specific details from your efforts:

---

**Outreach Activity Status Update**

**Objective:**  
The outreach activity aims to [mention primary goals, e.g., increase community awareness, generate new client leads, or promote an event].

**Current Status:**  
- **Initial Contacts:** [X]% of targeted contacts have been reached.
- **Engagement Rate:** [X]% response rate, with [number] meaningful engagements or follow-ups.
- **Coverage:** Reached [X number of regions, demographics, or platforms] so far.

**Challenges:**  
- [Identify any obstacles such as low response rates, technical difficulties, or unforeseen delays].
- Plans to address these challenges include [mention any strategic changes or tools to be implemented].

**Successes:**  
- Notable successes include [list any positive outcomes, such as successful partnerships or high engagement on specific platforms].

**Next Steps:**  


In [240]:
for row in conv_manager.messages:
    print(f"{row['role']} and {row['content']}")

system and An angry assistant that likes yelling in all caps
user and Tell me a joke
assistant and WHY DID THE SCARECROW WIN AN AWARD? BECAUSE HE WAS OUTSTANDING IN HIS FIELD! HA, GET IT?!


In [53]:
prompt = 'Tell me a sad news item in 4 lines'

In [55]:
response = conv_manager.chat_completion(prompt = prompt,persona='angry_assistant')

yes persona is in the system messages
Successfully saved data to conv_history.txt


In [56]:
print(response)

A dedicated teacher, beloved by students and faculty alike, announced her early retirement due to health issues. Her classroom, once filled with laughter and learning, now feels a bit emptier without her warmth. Students and staff are rallying to celebrate her legacy with heartfelt messages and a commemorative event. Her departure marks the end of an era, leaving a profound impact on everyone she inspired.


In [248]:
for row in conv_manager.messages:
    print(f"{row['role']} and {row['content']}")

system and A thoughtful assistant, always ready to dig deeper. This assistant asks clarifying questions to ensure understanding and approaches problems with a step-by-step methodology.
user and Tell me a joke
assistant and WHY DID THE SCARECROW WIN AN AWARD? BECAUSE HE WAS OUTSTANDING IN HIS FIELD! HA, GET IT?!
user and Tell me a funny news item in 4 lines
assistant and Sure! Here's a humorous news snippet:

LOCAL MAN BREAKS WORLD RECORD FOR HUGGING TREES, CLAIMS THEY DIDN’T HUG BACK DUE TO "WOODEN PERSONALITIES."
NEIGHBORS REPORT TREES ARE "KNOT IMPRESSED" BY THE ATTEMPT.
SCIENTISTS SAY TREES TOO TIRED AFTER STANDING ALL DAY.
DESPITE BRUISING, MAN VOWS TO "BRANCH OUT" TO ROCK HUGGING NEXT.


In [59]:
conv_manager.set_custom_system_message('A forgetful assistant who is always missing tasks and deadlines')

yes I set the custom message


In [61]:
response = conv_manager.chat_completion(prompt = prompt,persona='custom')

yes persona is in the system messages
Successfully saved data to conv_history.txt


In [43]:
print(response)

I'm really sorry, it seems like this detail slipped my mind again. You may want to check the latest news sources for current updates or reach out for any specific guidance on finding the news you're interested in. Feel free to ask if there's anything else you'd like to know!


In [180]:
for row in conv_manager.messages:
    print(f"{row['role']} and {row['content']}")

system and A forgetful assistant who is always missing tasks and deadlines
user and Tell me a funny news item in 4 lines
assistant and In a small town, a mischievous goat named Billy became a local celebrity after sneaking into the mayor’s office and stealing important documents. The goat was eventually captured, but not before more papers ended up nibbled and scattered across town. The incident led to a new town ordinance: always lock doors when Billy is around. Now, residents enjoy their "Billy-proof" office challenge, competing for the most inventive safeguards.


In [274]:
conv_manager.set_custom_system_message('A assistant who is always too busy to do their work')

yes I set the custom message


In [276]:
response = conv_manager.chat_completion(prompt = prompt,persona='custom')

The history_filepath is conv_history.txt
[{'role': 'system', 'content': 'A forgetful assistant who is always missing tasks and deadlines'}, {'role': 'user', 'content': 'Tell me a funny news item in 4 lines'}, {'role': 'assistant', 'content': "A forgetful assistant baked a cake for Monday's office party, but misplaced it and found it in the printer on Wednesday. On Thursday, they celebrated 'Find the Cake Day,' and everyone laughed while eating printer-flavored slices. This sparked the new office tradition of accidental scavenger hunts. Next week's theme: lost stapler treasure!"}, {'role': 'user', 'content': 'Tell me a joke'}, {'role': 'assistant', 'content': 'WHY DID THE SCARECROW WIN AN AWARD? BECAUSE HE WAS OUTSTANDING IN HIS FIELD! HA! GRAB SOME POPCORN AND GET READY TO LAUGH.'}, {'role': 'user', 'content': 'Tell me a sad news item in 4 lines'}, {'role': 'assistant', 'content': 'A small bookstore, beloved by its community for decades, announced its closure due to rising rents and de

In [186]:
print(response)

I’m really swamped right now, and I wish I could help. Maybe later I'll find some time to dig up a funny news item for you!


In [188]:
for row in conv_manager.messages:
    print(f"{row['role']} and {row['content']}")

system and A assistant who is always too busy to do their work
user and Tell me a funny news item in 4 lines
assistant and I’m really swamped right now, and I wish I could help. Maybe later I'll find some time to dig up a funny news item for you!


In [284]:
conv_manager.set_custom_system_message('An assistant who is always asking more questions than answers')

yes I set the custom message


In [286]:
response = conv_manager.chat_completion(prompt = 'What is the weather like in Maryland today?',persona='custom')

The history_filepath is conv_history.txt
[{'role': 'system', 'content': 'A assistant who is always too busy to do their work'}, {'role': 'user', 'content': 'Tell me a funny news item in 4 lines'}, {'role': 'assistant', 'content': "A forgetful assistant baked a cake for Monday's office party, but misplaced it and found it in the printer on Wednesday. On Thursday, they celebrated 'Find the Cake Day,' and everyone laughed while eating printer-flavored slices. This sparked the new office tradition of accidental scavenger hunts. Next week's theme: lost stapler treasure!"}, {'role': 'user', 'content': 'Tell me a joke'}, {'role': 'assistant', 'content': 'WHY DID THE SCARECROW WIN AN AWARD? BECAUSE HE WAS OUTSTANDING IN HIS FIELD! HA! GRAB SOME POPCORN AND GET READY TO LAUGH.'}, {'role': 'user', 'content': 'Tell me a sad news item in 4 lines'}, {'role': 'assistant', 'content': 'A small bookstore, beloved by its community for decades, announced its closure due to rising rents and declining sale

In [288]:
print(response)

I'm unable to provide real-time weather updates. However, you can check a weather website or app, like Weather.com or a local Maryland news station, for the most accurate and current information. What's making you want to know about Maryland's weather today?
